In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import psycopg2.extras as extras
from google.cloud import bigquery
from google.api_core.exceptions import BadRequest
from dotenv import dotenv_values
from datetime import datetime,date,timedelta

# eeeoe 07-07-203 have 1 but bq=0

# Init Params

In [3]:
isDailyMode=True
projectId='pongthorn'
dataset_id='SMartDW'


datetime_imported=datetime.now()

env_path=r'D:\PythonDev\MyQuantFinProject\SMart-AI\.env'
config = dotenv_values(dotenv_path=env_path)

# the first load and it became train/test dat
imported_from='2022-01-01'
day_imported='2023-07-07'



# Set Date  to ingest data along with given mode
## Unseen data as Daily Mode : Yesterday data
## Train data as ML Mode : Given period

In [5]:
if isDailyMode:
    today = date.today()   
    
    day_imported = today - timedelta(days = 1)
    day_imported=day_imported.strftime('%Y-%m-%d')
    print(f"DailyMode={isDailyMode} :  {day_imported}" )
else:
    imported_to = datetime.strptime(day_imported,'%Y-%m-%d')
    imported_to = imported_to + timedelta(days = 1)
    imported_to=imported_to.strftime('%Y-%m-%d')
    print(f"DailyMode={isDailyMode} : From {imported_from} To {imported_to}" )
    

dateTimeCol='incident_datetime'
dateCol='date'

countCol='count_incident'

tf='D'
bigger_tf='W-SUN'


DailyMode=True :  2023-09-03


# BigQuery

In [6]:
table_id = f"{projectId}.{dataset_id}.daily_incident"
# create table first
client = bigquery.Client(project=projectId)
def insertDataFrameToBQ(df_trasns):
    try:
        if isDailyMode==True:
            job_config = bigquery.LoadJobConfig(
                write_disposition="WRITE_APPEND",
            )
        else:
            job_config = bigquery.LoadJobConfig(
                write_disposition="WRITE_TRUNCATE",
            )

        job = client.load_table_from_dataframe(
            df_trasns, table_id, job_config=job_config
        )
        job.result()  # Wait for the job to complete.
        print("Total ", len(df_trasns), "Imported closed incident to bigquery successfully")

    except BadRequest as e:
        print("Bigquery Error\n")
        print(e) 


# Postgres

In [92]:
def get_postgres_conn():
 try:
  conn = psycopg2.connect(
        database=config['DATABASES_NAME'], user=config['DATABASES_USER'],
      password=config['DATABASES_PASSWORD'], host=config['DATABASES_HOST']
     )
  return conn

 except Exception as error:
  print(error)      
  raise error
def list_data(sql,params,connection):
 df=None   
 with connection.cursor() as cursor:
    
    if params is None:
       cursor.execute(sql)
    else:
       cursor.execute(sql,params)
    
    columns = [col[0] for col in cursor.description]
    dataList = [dict(zip(columns, row)) for row in cursor.fetchall()]
    df = pd.DataFrame(data=dataList) 
 return df 

# Create SQL Statement for  pulling data from  Postgres


In [7]:
if isDailyMode==False:
    sql=f"""
    select Date({dateTimeCol}) as {dateCol},count(*) as {countCol}  from app_incident
    where {dateTimeCol}>='{imported_from}' and incident_datetime<'{imported_to}'
    group by   Date({dateTimeCol}) order by Date({dateTimeCol})
    """
else:
    sql=f"""
    select count(*) as {countCol}  from app_incident
    where  Date({dateTimeCol})='{day_imported}'
    """
print(isDailyMode)    
print(sql)


True

    select count(*) as count_incident  from app_incident
    where  Date(incident_datetime)='2023-09-03'
    


In [94]:
df_incident=list_data(sql,None,get_postgres_conn())
if isDailyMode:
   df_incident[dateCol] =day_imported
   df_incident=df_incident[[dateCol,countCol]] 
df_incident[dateCol]=pd.to_datetime(df_incident[dateCol], format='%Y-%m-%d')
print(df_incident.info()) 
print(df_incident.head(10))
print(df_incident.tail(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            1 non-null      datetime64[ns]
 1   count_incident  1 non-null      int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 144.0 bytes
None
        date  count_incident
0 2023-07-08               0
        date  count_incident
0 2023-07-08               0


# Transforam data and Ingest to BQ

In [95]:
try: 
 if isDailyMode==False:
 
    print("Create date range to get prepared for build time-series dataframe")
    from_date=df_incident[dateCol].min().strftime('%Y-%m-%d')
    to_date=day_imported
    # to_date=df_incident[dateCol].max().strftime('%Y-%m-%d')
    period_index=pd.date_range(start=from_date,end=to_date, freq='D')
    print(period_index)
    dfDate=pd.DataFrame(index=period_index,columns=['date'])
    dfDate.reset_index(drop=False,inplace=True)
    dfDate['date']=dfDate['index']
    dfDate.drop(columns=['index'],inplace=True)
    print(dfDate.info())
    print(dfDate.head())
    print(dfDate.tail())

    print("Merge data as time series data")
    df=dfDate.merge(df_incident,how='left',left_on='date',right_on=dateCol)
    df=df[['date',countCol]]
    df.fillna(value=0,inplace=True)
    df['datetime_imported']=datetime_imported
    # df=df.set_index('date')
    print(df.info())
    print(df.tail(10))

    
    df.to_csv('DailyIncident.csv',index=False)
    insertDataFrameToBQ(df)
 else:
    df_incident['datetime_imported']=datetime_imported
    insertDataFrameToBQ(df_incident)
    
except Exception as ex:
     raise ex
        


Total  1 Imported closed incident to bigquery successfully


In [76]:
# dfXYZ=df.set_index('date').resample(bigger_tf).sum()
# print(dfXYZ.info())
# print(dfXYZ)

# import plotly.express as px

# fig = px.line(dfXYZ, y=countCol, title=f'{bigger_tf}-{countCol}')
# fig.show()
